In [1]:
%config IPCompleter.greedy = True
%autosave 60

Autosaving every 60 seconds


## 1. NLP

### Cargar archivo

In [2]:
import pandas as pd
import os
ruta_archivo = os.path.join("imdb_dataset.csv")
df_criticas = pd.read_csv(ruta_archivo, encoding='iso-8859-2').sample(2000, replace=False)
#df_criticas = pd.read_csv(ruta_archivo, encoding='iso-8859-2')
df_criticas

,Review,Label
45630,Red Skelton was still another major star who m...,pos
34232,There's so little here of the fantastic Anne R...,neg
1902,I am not a fan of the original book but was ex...,neg
17384,"In the world of ""shorts"" (most of which aren't...",pos
29334,The first 30min of the flick was choppy and ha...,neg
...,...,...
16247,Grand Champion is a bit old fashion at first g...,pos
30041,Hollywood now has officially gone too far and ...,neg
2665,I viewed the first two nights before coming to...,neg
25463,This would've been a *great* silent film. The ...,neg


### Palabras de parada

In [3]:
import nltk
import string
# nltk.download('punkt')
# nltk.download('stopwords')
palabras_de_parada_ingles = set(nltk.corpus.stopwords.words('english') + list(string.punctuation) + ['...', '..'])
palabras_de_parada_ingles

{'!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '..',
 '...',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 '

### Normalizar el texto

In [4]:
# LimpiadorTexto es una clase que nosotros creamos.
# Utilizamos la funcion normalizar aquí, en el notebook. y también en la API.
# Por tanto, para no tener código repetido, abstraímos la lógica a una clase externa.
from LimpiadorTexto import LimpiadorTexto
limpiador = LimpiadorTexto(palabras_de_parada_ingles)
df_criticas = df_criticas.apply(limpiador.normalizar_fila, axis=1)
df_criticas

,Review,Label
45630,red skelton still another major star made tran...,pos
34232,little fantastic anne rice book makes sense ch...,neg
1902,fan original book expecting see better adaptat...,neg
17384,world shorts film gem quiet concise peek world...,pos
29334,first 30min flick choppy hard know going unles...,neg
...,...,...
16247,grand champion bit old fashion first glance an...,pos
30041,hollywood officially gone far really hope trav...,neg
2665,viewed first two nights coming imdb looking ac...,neg
25463,would great silent film acting really good lea...,neg


### Obtener el vocabulario del problema

In [5]:
def obtener_vocabulario_problema(df_criticas_normalizadas):
    cuerpo_texto_normalizado = ' '.join(df_criticas_normalizadas['Review'].tolist())
    vocabulario_problema = cuerpo_texto_normalizado.split()
    vocabulario_ordenado = sorted(set(vocabulario_problema))
    return vocabulario_ordenado

def obtener_vocabulario_problema_y_posicion(vocabulario_del_problema_ordenado):
    vocabulario_y_posicion = {}
    for i, token in enumerate(vocabulario_del_problema_ordenado):
        vocabulario_y_posicion[token] = i
    return vocabulario_y_posicion

In [6]:
vocabulario_problema = obtener_vocabulario_problema_y_posicion(obtener_vocabulario_problema(df_criticas))
len(vocabulario_problema)

30372

### One Hot Encoding

In [7]:
import numpy as np

def obtener_one_hot_vector(critica, vocabulario_problema_y_posicion):
    one_hot_vector = np.zeros(len(vocabulario_problema_y_posicion), dtype=int)
    for token in critica.split():
        one_hot_vector[vocabulario_problema_y_posicion[token]] = 1
    return one_hot_vector

In [8]:
one_hots = []
indices = []
for indice, fila in df_criticas.iterrows():
    one_hot = obtener_one_hot_vector(fila['Review'], vocabulario_problema)
    indices.append(indice)
    one_hots.append(one_hot)

## 2. Crear y entrenar el modelo predictivo

In [9]:
# se cambia el nombre de la variable 'one_hots' para mas claridad
X = one_hots
Y = df_criticas['Label'].ravel()

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [11]:
import gc
# Liberar memoria RAM
lst = [df_criticas]
del df_criticas, one_hots, X, Y
del lst
gc.collect()

140

In [12]:
from sklearn.linear_model import LogisticRegression
clasificador_reg_log = LogisticRegression(random_state=0, solver='liblinear') #Falta añadir el solver del magister...
clasificador_reg_log.fit(X_train, Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [13]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
predicciones = clasificador_reg_log.predict(X_test)
metricas = {
    'accuracy': accuracy_score(Y_test, predicciones),
    'ṕrecision': precision_score(Y_test, predicciones, pos_label='pos'),
    'recall': recall_score(Y_test, predicciones, pos_label='pos'),
    'f1': f1_score(Y_test, predicciones, pos_label='pos')
}
print(metricas)

{'accuracy': 0.8133333333333334, 'ṕrecision': 0.8513513513513513, 'recall': 0.7875, 'f1': 0.818181818181818}


### Guardar el clasificador como un archivo

In [14]:
import pickle
ruta_archivo_clasificador = os.path.join('clasificador-regresion-logistica.pkl')
archivo_clasificador = open(ruta_archivo_clasificador, 'wb')
pickle.dump(clasificador_reg_log, archivo_clasificador)
archivo_clasificador.close()

### Guardar el vocabulario del problema como un archivo

In [15]:
ruta_archivo_vocabulario = os.path.join('vocabulario-problema.pkl')
archivo_vacabulario = open(ruta_archivo_vocabulario, 'wb')
pickle.dump(vocabulario_problema, archivo_vacabulario)
archivo_vacabulario.close()

### Guardar las palabras de parada como un archivo

In [16]:
ruta_archivo_palabras_parada = os.path.join('palabras-parada.pkl')
archivo_palabras_parada = open(ruta_archivo_palabras_parada, 'wb')
pickle.dump(palabras_de_parada_ingles, archivo_palabras_parada)
archivo_palabras_parada.close()